In [1]:
# 事前に "gender_submission.csv"  "train.csv" "test.csv" を準備すること

In [2]:
import warnings

warnings.filterwarnings("ignore")

## 1. 問題の定義

### 解くべき問題は何か？
- タイタニック号の乗船客の生存と死亡を予測すること

### どういう問題として処理するか？
- 教師あり学習
- 2クラス分類問題


## 2. データの取得・観察


In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train = pd.read_csv("train.csv")

df_test = pd.read_csv("test.csv")

## 3. 評価指標と評価プロトコル

### 評価指標
- 正答率(Accuracy)を利用する

### 評価プロトコル
- 交差検証を利用してみる

## 4. データの前処理・特徴量エンジニアリング
- 別の特徴量も試してみる

## 5. モデルの訓練・評価

### モデルの訓練 と モデルの評価
- 機械学習アルゴリズム は LogisticRegressionのみで進める

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

In [6]:
# まずはParchのみで交差検証
X = df_train[["Parch"]]
y = df_train[["Survived"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

array([0.6119403 , 0.61940299, 0.61940299, 0.61654135, 0.61654135])

In [7]:
# いまいちなので、特徴量を変えてみる(部屋の等級が高いほうが優先されたのでは？)
X = df_train[["Pclass"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

array([0.68656716, 0.67164179, 0.67164179, 0.67669173, 0.65413534])

In [8]:
# 性別情報を追加してみる(カテゴリ化)
X = df_train[["Pclass"]]
df_sex = pd.get_dummies(df_train["Sex"])

X = pd.concat((df_train[["Pclass"]], df_sex), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

cross_val_score(LogisticRegression(), X_train, y_train, cv=5)

array([0.76119403, 0.85074627, 0.73880597, 0.76691729, 0.81954887])

## 6. モデルのチューニング
- 等級 と 性別 の2つの特徴量で進める
- あとはLogisticRegressionのパラメータCをグリッドサーチで調整する

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)

grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_params_)

0.7889221556886228
{'C': 0.01}


## kaggleへのsubmit用データをつくる

In [10]:
X_new = pd.concat(
    (df_test[["Pclass"]], pd.get_dummies(df_test["Sex"])), axis=1)


pred = grid.predict(X_new)

In [11]:
# sumission用に整形

df_submission = pd.DataFrame(columns=["PassengerId", "Survived"])
df_submission["PassengerId"] = df_test["PassengerId"]
df_submission["Survived"] = pred

In [12]:
df_submission.to_csv("02_submision.csv", index=False)